In [3]:
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Search for any field in JSON object

In [4]:
def get_field_values(json_obj, field_name):
    """
    Returns a list of all values of the specified field in the JSON object
    with any level of nesting.
    """
    result = []
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if key == field_name:
                result.append(value)
            else:
                result.extend(get_field_values(value, field_name))
    elif isinstance(json_obj, list):
        for item in json_obj:
            result.extend(get_field_values(item, field_name))
    return result

### Expected ELO calculation

In [5]:
def expected_result(this, opponent):
    return 1 / (1 + 10 ** ((opponent - this) / 400))

### This part scrapes the data into a df and provides intital ELO <b>---outdated---</b>

In [22]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

In [ ]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

all_players = []
matchcount = 0

for i in range(1, len(data)):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    matchcount += 1
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        win = 1 if get_field_values(player, 'win')[0] else 0
        loss = 0 if get_field_values(player, 'win')[0] else 1
        
        new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

        # check if the player is already in the DataFrame
        if pid in playersdf['id'].tolist():
            # increment the number of matches played for the existing player
            playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
            playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
            playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
        else:
            # add the new player to the DataFrame
            playersdf = playersdf.append(new_player, ignore_index=True)
            
        
        
playersdf

### Adds players to df and calculates player ELO

In [24]:
with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

all_players = []
matchcount = 0

for i in range(1, len(data)):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    matchcount += 1
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        win = 1 if get_field_values(player, 'win')[0] else 0
        loss = 0 if get_field_values(player, 'win')[0] else 1
        
        new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

        # check if the player is already in the DataFrame
        if pid in playersdf['id'].tolist():
            # increment the number of matches played for the existing player
            playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
            playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
            playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
        else:
            # add the new player to the DataFrame
            playersdf = playersdf.append(new_player, ignore_index=True)
        
    winners = get_field_values(participants, 'win')
    if not any(winners):
        teamAELO = 0.0
        teamBELO = 0.0
        for n in range(5):
            teamAELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        for n in range(5, 10):
            teamBELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        expA = expected_result(teamAELO, teamBELO)
        deltaAELO = k * (0.5 - expA)
        expB = expected_result(teamBELO, teamAELO)
        deltaBELO = k * (0.5 - expB)
        print('-------------------------------------')
        print('\n', i, ' A ELO: ', teamAELO, ' B ELO: ', teamBELO)
        print(i, ' deltaAELO: ', deltaAELO, ' deltaBELO: ', deltaBELO)  
        
        for n in range(5):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaAELO
        for n in range(5, 10):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaBELO
        
        
    else:
        winnerteam = []
        loserteam = []
        winnerELO = 0.0
        loserELO = 0.0
        for n, w in enumerate(winners):
            if w:
                winnerteam.append(participants[0][n])
                winnerELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
            else:
                loserteam.append(participants[0][n])
                loserELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5

        k = 100.0
        expected_winner = expected_result(winnerELO, loserELO)
        delta_winnerELO = k * (1 - expected_winner)
        expected_loser = expected_result(loserELO, winnerELO)
        delta_loserELO = k * (0 - expected_loser)
        print('\n', i, ' WinnerELO: ', winnerELO, ' loserELO: ', loserELO)
        print(i, ' Delta_WinnerELO: ', delta_winnerELO, ' delta_loserELO: ', delta_loserELO)
    
        for n, w in enumerate(winners):
            if w:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_winnerELO
            else:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_loserELO
        

        
playersdf


 2  WinnerELO:  981.3707797938853  loserELO:  950.0
2  Delta_WinnerELO:  45.497610794540336  delta_loserELO:  -45.497610794540336

 3  WinnerELO:  990.4251842672484  loserELO:  937.7328900163947
3  Delta_WinnerELO:  42.47457827948783  delta_loserELO:  -42.47457827948783

 4  WinnerELO:  1026.131966640428  loserELO:  1082.18422011746
4  Delta_WinnerELO:  57.99730371265266  delta_loserELO:  -57.99730371265266

 5  WinnerELO:  1084.1292703530803  loserELO:  1024.1869164048076
5  Delta_WinnerELO:  41.458185721049915  delta_loserELO:  -41.45818572104991

 76  WinnerELO:  1125.5874560741302  loserELO:  1008.3905061946693
76  Delta_WinnerELO:  33.7458675125257  delta_loserELO:  -33.74586751252569

 78  WinnerELO:  974.6446386821436  loserELO:  942.5545122839553
78  Delta_WinnerELO:  45.39494720501666  delta_loserELO:  -45.39494720501667

 96  WinnerELO:  982.7287306837575  loserELO:  1026.8683905884257
96  Delta_WinnerELO:  56.31825155178413  delta_loserELO:  -56.31825155178413

 97  WinnerE


 504  WinnerELO:  1281.3695098798114  loserELO:  985.7788728606058
504  Delta_WinnerELO:  15.426208837090027  delta_loserELO:  -15.426208837090027

 505  WinnerELO:  1072.7519647247082  loserELO:  996.8970892355567
505  Delta_WinnerELO:  39.25381216497254  delta_loserELO:  -39.25381216497254

 506  WinnerELO:  1133.7715816694615  loserELO:  1045.5916347099064
506  Delta_WinnerELO:  37.57552359299234  delta_loserELO:  -37.57552359299235

 507  WinnerELO:  1112.0057768896804  loserELO:  932.9993722231868
507  Delta_WinnerELO:  26.299809322546785  delta_loserELO:  -26.29980932254678

 508  WinnerELO:  1146.50331011884  loserELO:  1008.0161111169143
508  Delta_WinnerELO:  31.062555529549364  delta_loserELO:  -31.062555529549368

 509  WinnerELO:  970.3526640235159  loserELO:  1030.0686499279968
509  Delta_WinnerELO:  58.510184542984526  delta_loserELO:  -58.510184542984526

 510  WinnerELO:  1554.5125699623181  loserELO:  1103.8492140747742
510  Delta_WinnerELO:  6.951089831913892  delta_


 568  WinnerELO:  1627.3567118229396  loserELO:  1201.4362466295138
568  Delta_WinnerELO:  7.930736255675475  delta_loserELO:  -7.930736255675479

 569  WinnerELO:  1091.0095974758701  loserELO:  1282.4907916101995
569  Delta_WinnerELO:  75.06822660316394  delta_loserELO:  -75.06822660316392

 570  WinnerELO:  1376.360630330496  loserELO:  1144.8984263085515
570  Delta_WinnerELO:  20.876206754669713  delta_loserELO:  -20.87620675466971

 571  WinnerELO:  1348.9705943917736  loserELO:  1397.2368370851657
571  Delta_WinnerELO:  56.90172851647903  delta_loserELO:  -56.90172851647903

 572  WinnerELO:  1316.6689675454754  loserELO:  1166.0778240790341
572  Delta_WinnerELO:  29.590552673139648  delta_loserELO:  -29.59055267313964

 573  WinnerELO:  1612.317433077305  loserELO:  1346.2595202186153
573  Delta_WinnerELO:  17.77666558922075  delta_loserELO:  -17.776665589220755

 574  WinnerELO:  1405.8723229082525  loserELO:  1340.3351085686863
574  Delta_WinnerELO:  40.67873103736771  delta_

,id,ELO,matches_played,wins,losses
0,eSENkVd2RZz7UPvDJV4DUGdEesrck79UoqufrU_zbnvzzU...,904.502389,2.0,0.0,2.0
1,tjn0xgZp-NXT7nnqUkUyFij-WpkuyW0C48R-BueCtsJ96y...,904.502389,2.0,0.0,2.0
2,cWegWX9cAlTTu89tJqlAVTx6UBaXwFvtlZ_vfec9BVCyxw...,904.502389,2.0,0.0,2.0
3,4UXXNNnYVIw7-Yy1fUDX4g-HcXCWbg9seluDuuvZYyOjh5...,904.502389,2.0,0.0,2.0
4,COFiGhpicYc5NBOQqt76w4byLgqIBo3IbpalTnl7fyaPTC...,904.502389,2.0,0.0,2.0
...,...,...,...,...,...
384,7nciBasYULX3ZfR7zONSSmDjhqj9F1g3Br1ElxKpFJZIwq...,934.049903,10.0,4.0,6.0
385,K7PZGV21WiS6YNYe6nXZ498ZiS9jKeun-C3J7X1npMbtpK...,1217.133410,20.0,16.0,4.0
386,NzuXdkdnH86tpmVYluykXL4ywJ2QF-aT6EV4t_MhTSLcWF...,1217.133410,20.0,16.0,4.0
387,5NdpwQH95lFFDULCBDI81zLbIOOE8V2iZHRJ0yE94vEWDF...,1170.920744,16.0,10.0,6.0


### Seeing how the ranking changes over many played rounds

In [7]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

In [8]:
plotdata = []


with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

for p in range(20):
    all_players = []

    for i in range(1, len(data)):
        participants = get_field_values(data[i], 'participants')
        if len(participants[0]) != 10:
            continue

        for player in participants[1]:
            pid = get_field_values(player, 'puuid')[0]
            win = 1 if get_field_values(player, 'win')[0] else 0
            loss = 0 if get_field_values(player, 'win')[0] else 1

            new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

            # check if the player is already in the DataFrame
            if pid in playersdf['id'].tolist():
                # increment the number of matches played for the existing player
                playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
                playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
                playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
            else:
                # add the new player to the DataFrame
                playersdf = playersdf.append(new_player, ignore_index=True)

        winners = get_field_values(participants, 'win')
        if not any(winners):
            #TODO draw needs to be handled
            continue

        winnerteam = []
        loserteam = []
        winnerELO = 0.0
        loserELO = 0.0
        for n, w in enumerate(winners):
            if w:
                winnerteam.append(participants[0][n])
                winnerELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
            else:
                loserteam.append(participants[0][n])
                loserELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5

        k = 32.0
        expected_winner = expected_result(winnerELO, loserELO)
        delta_winnerELO = k * (1 - expected_winner)
        expected_loser = expected_result(loserELO, winnerELO)
        delta_loserELO = k * (0 - expected_loser)

        for n, w in enumerate(winners):
            if w:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_winnerELO
            else:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_loserELO
    
    plotdata.append(playersdf.loc[208, 'ELO'])

In [19]:
playersdf = playersdf.sort_values(by = 'ELO', ascending = False)

In [20]:
playersdf[['id', 'ELO', 'matches_played']].to_json('import.json', orient='records')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(20), plotdata)

In [ ]:
exp = pd.DataFrame({'x':range(20), 'y':plotdata})
exp.to_csv('exp.csv')

In [ ]:
with open('lolMatch.json', encoding="utf-8") as f:
    data = json.load(f)

participants = get_field_values(data[2], 'participants')
player = participants[1][0]
pid = get_field_values(player, 'puuid')[0]
playerstats = pd.DataFrame(player)
playerstats.to_csv('playerdata.csv')
playerstats.drop('styles', inplace = True)
playerstats.drop('perks', axis = 1, inplace = True)
playerstats

In [ ]:
for i in range(1, 5):
    participants = get_field_values(data[i], 'participants')
    if len(participants[0]) != 10:
        continue
    
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        playerstats = pd.DataFrame(player)
        playerstats.head()
        
    player = participants[1][0]
    pid = get_field_values(player, 'puuid')[0]
    playerstats = pd.DataFrame(player)
    playerstats.head()

In [26]:
import pandas as pd
playersdf = pd.DataFrame({'id': [], 'ELO': [], 'matches_played': [], 'wins': [], 'losses': []})

In [89]:
import os

directory_path = "C:/Users/kofinandi/Desktop/k-h_hackathon/jupyter_analysis/matches"

file_names = os.listdir(directory_path)

# Loop through each file name in the list and open/read the file
for i, file_name in enumerate(file_names[0:3000]):
    if (i % 200 == 0):
        print(i, ' json processed')
    # Construct the full path to the file by joining the directory path and file name
    file_path = os.path.join(directory_path, file_name)
    
    # Open the file and read its contents
    with open(file_path, encoding="utf-8") as f:
        data = json.load(f)

    participants = get_field_values(data, 'participants')
    if len(participants[0]) != 10:
        continue
    matchcount += 1
    for player in participants[1]:
        pid = get_field_values(player, 'puuid')[0]
        win = 1 if get_field_values(player, 'win')[0] else 0
        loss = 0 if get_field_values(player, 'win')[0] else 1
        
        new_player = {'id': pid, 'ELO': 1000, 'matches_played': 1, 'wins': win, 'losses': loss}

        # check if the player is already in the DataFrame
        if pid in playersdf['id'].tolist():
            # increment the number of matches played for the existing player
            playersdf.loc[playersdf['id'] == new_player['id'], 'matches_played'] += new_player['matches_played']
            playersdf.loc[playersdf['id'] == new_player['id'], 'wins'] += new_player['wins']
            playersdf.loc[playersdf['id'] == new_player['id'], 'losses'] += new_player['losses']
        else:
            # add the new player to the DataFrame
            playersdf = playersdf.append(new_player, ignore_index=True)
        
    winners = get_field_values(participants, 'win')
    if not any(winners):
        teamAELO = 0.0
        teamBELO = 0.0
        for n in range(5):
            teamAELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        for n in range(5, 10):
            teamBELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
        expA = expected_result(teamAELO, teamBELO)
        deltaAELO = k * (0.5 - expA)
        expB = expected_result(teamBELO, teamAELO)
        deltaBELO = k * (0.5 - expB) 
        
        for n in range(5):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaAELO
        for n in range(5, 10):
            playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += deltaBELO
        
        
    else:
        winnerteam = []
        loserteam = []
        winnerELO = 0.0
        loserELO = 0.0
        for n, w in enumerate(winners):
            if w:
                winnerteam.append(participants[0][n])
                winnerELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5
            else:
                loserteam.append(participants[0][n])
                loserELO += playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'].values[0] / 5

        k = 100.0
        expected_winner = expected_result(winnerELO, loserELO)
        delta_winnerELO = k * (1 - expected_winner)
        expected_loser = expected_result(loserELO, winnerELO)
        delta_loserELO = k * (0 - expected_loser)
    
        for n, w in enumerate(winners):
            if w:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_winnerELO
            else:
                playersdf.loc[playersdf['id'] == (participants[0][n]), 'ELO'] += delta_loserELO
        

        
playersdf

0  json processed
200  json processed
400  json processed
600  json processed
800  json processed
1000  json processed
1200  json processed
1400  json processed
1600  json processed
1800  json processed
2000  json processed
2200  json processed
2400  json processed
2600  json processed
2800  json processed


,id,ELO,matches_played,wins,losses
0,Xp8x-GYhSz3A7by9sRP24AnTVBwSi4g1_V77TewTFIhkpq...,1224.674710,6.0,6.0,0.0
1,ly0n3iMHd4KWn8YphydQzDVUSgNkz0XZMzNPNbphnj_wWG...,1584.259460,301.0,187.0,114.0
2,IJTLRo8VTOQgoYsuR1vHRPb1lWRlWger9k7mnXgvfOM4-u...,1088.835450,3.0,3.0,0.0
3,e3G7VRFHaxJO6s24ACOiXjmw9bhXs-0bQBZGODqzBLt506...,958.627883,6.0,3.0,3.0
4,tAS8P0LcwFmOnXeyaQKf-WR4RGpKItDw1kB0sC4-NMYiOW...,937.022800,6.0,3.0,3.0
...,...,...,...,...,...
37736,Mma0iYZaqDNxwMWwB0tnt1MKda9SOP8EH8zYDcKZRbUhau...,1053.001966,1.0,1.0,0.0
37737,hD-an_BfwB021k-s4D5jR_hwtPVTsot9C9wGUrvm1R8iKI...,946.998034,1.0,0.0,1.0
37738,IEhgo0iL_1jQTAq_UrFAEQ1Af54FgFfHNcgcloMADo4frp...,946.998034,1.0,0.0,1.0
37739,pR0lDCpcoFOVq4YAxUH0VdVCJrfWvFp9gIKqwHHYR24U5R...,946.998034,1.0,0.0,1.0


In [90]:
st4 = playersdf

In [35]:
manymatches = playersdf[playersdf['matches_played'] > 10]
manymatches

,id,ELO,matches_played,wins,losses
1,ly0n3iMHd4KWn8YphydQzDVUSgNkz0XZMzNPNbphnj_wWG...,1552.954860,199.0,123.0,76.0
14,0FQdIWHvNpvFQrW0YNwCsHmO3kkDy_GTI4flyBhCwaECi2...,1274.382775,14.0,10.0,4.0
20,zBf86Hk2Hr9IXYX7yDWut6CEvnw4OFSQlp9ZMxOOg06qXk...,1006.093038,20.0,10.0,10.0
28,IFFCEO00VJPxKs75bjkr7f7UOVc4ZprcwA9Z0zk8kTDczu...,1516.505612,32.0,24.0,8.0
40,G74bFhTyIOF5HBCWAYN6a6uoJT6ElRY2PeB5Zcu31LhyH7...,772.533547,14.0,4.0,10.0
...,...,...,...,...,...
17916,J3EJunfJXh-EyVZKA0JDBn4BFgXmWcKomrMBkyvBtZGWps...,1305.741807,17.0,12.0,5.0
18205,d4A0YiMh0YPoWvcCO8YE8vTUIJBiCOIV2IfxQVVPoh9Pfd...,1055.783673,12.0,7.0,5.0
18499,AzsnFKveuw4HPIRmLBz2HKwHSuWRlOnszYFao2Kscd0O_H...,921.370890,13.0,6.0,7.0
18503,u2R-BNHH0IYpeu9WJqFWdv85JX8WxVokPekaST8QTVa2uD...,1197.272832,11.0,8.0,3.0


In [36]:
manymatches.sort_values(by = 'ELO')

,id,ELO,matches_played,wins,losses
2731,4n_XTUWsh56di0pcApmsvGX75C-VOz1kbKXU05J8Lhjk1f...,488.991505,48.0,16.0,32.0
359,mf8eRXWVJj6G-RNDOUzuXNpmG0EZsdo2WJ8l8MxZDuhBef...,604.106005,14.0,1.0,13.0
3157,UyJfS3rXEuMZqCrY7szQXnFM-sFhBuL2mxuSBCQDKGQgEH...,638.058491,17.0,4.0,13.0
14651,om1zWkLlFgbvv6EW-dDla9A2ahND95OzJ9y2ekvNdi26e-...,678.067897,12.0,3.0,9.0
132,r4vh6i02r7mUU2sYWhQcRIicKrH2fITVV_n2RGZOLA8SzZ...,682.801172,14.0,3.0,11.0
...,...,...,...,...,...
3390,Oj_TRuDzVxMf9gM1t0zyKeEHv5Q1gKzaMTzpXuencht8Qu...,1626.682553,23.0,19.0,4.0
7600,vHS4oyijL1nqSTs5C8iT-ZdPh-aZY1DeqcqrR0sTGenEsn...,1640.941761,102.0,69.0,33.0
5826,EatmToZiBqRnc3gW_fI8RyHokSgpE7xolcLV0YXgxVqGa7...,1681.962510,73.0,54.0,19.0
4642,XXpYzb6TNH3GYwSu5j2asFryxsxjeoPEuksJc9kwapZUkp...,1753.639030,107.0,80.0,27.0


In [42]:
playersdf.to_csv('iter8963.csv', index=False)

In [43]:
st1 = pd.read_csv('iter4000.csv')
st2 = pd.read_csv('iter6500.csv')
st3 = pd.read_csv('iter8963.csv')

In [44]:
st3.sort_values(by = 'ELO')

,id,ELO,matches_played,wins,losses
2547,oP14B6CSyIMex_iHzKuytInbvY_KnWm6vt-8NfoHsEJO9w...,559.603607,46.0,17.0,29.0
18861,d4jd9BsCmHQ39JBW3Q_l6i7kzX-IhpJdwnEbn-PE1Qh8NQ...,565.841433,26.0,8.0,18.0
32795,B69ODRi5zZsjJr_t-8WFOX2WxAPWbYe_L61ElASQ2E53su...,595.251848,34.0,11.0,23.0
20363,HhiNChzUz-rr-8ubYyc6uhXSHeYkZwTQ-L0lm_BdH_PCfG...,599.708118,16.0,4.0,12.0
10977,lDcT-pFlvXC0cxTptu5gmQpuTs60XuBWre17HDgIT2AJJS...,604.373253,11.0,2.0,9.0
...,...,...,...,...,...
16286,uboFqtccfxG2RVuvsrhPYMDcibRBxvRhz1JIrbE2D4RB8y...,1679.875595,116.0,74.0,42.0
345,5fcnxO32AOrnCL6GJqEUsFcDkgGCUdQK4oFi_oGSdtn5DU...,1726.096138,80.0,61.0,19.0
16445,tQW03KY3EiaFeNRxf_jGkur_9JCNPTQleq9YizxhzMPDhq...,1733.105916,99.0,65.0,34.0
13708,e5EfkZKRbASb7ccr7lw08U5qdoEU27Shd_srYxDRMy46VS...,1854.422691,146.0,108.0,38.0


In [50]:
st3[st3['matches_played'] > 150]

,id,ELO,matches_played,wins,losses
1,ly0n3iMHd4KWn8YphydQzDVUSgNkz0XZMzNPNbphnj_wWG...,1624.250746,201.0,125.0,76.0
577,TUV8HtyhE5dahkWTLzY6rR4wQc1Q2gfVhL1tu4VQrmCOiP...,1552.290767,198.0,132.0,66.0
1168,EENgG2qN3KH1ngLqrMYtLu4B0_dc1KNABHGkyfPQ5RRSOW...,1242.311700,171.0,78.0,93.0
3383,t0sKhAPAUcRuPW9A_gIZw_BAanp9nwNNasj8OI9LtT-Wwt...,1340.850447,194.0,120.0,74.0
3498,NlXZJsK3dr4L7j_YBeEmVpTCCl4P_-ZZKENXos80y9SpKL...,1523.955655,195.0,129.0,66.0


In [51]:
names = pd.read_csv('summoners.csv')

In [56]:
names = names[['puuid', 'summoner_name']]

In [57]:
names

,puuid,summoner_name
0,t0sKhAPAUcRuPW9A_gIZw_BAanp9nwNNasj8OI9LtT-Wwt...,Δ Desperate
1,tQW03KY3EiaFeNRxf_jGkur_9JCNPTQleq9YizxhzMPDhq...,Graaland
2,XXpYzb6TNH3GYwSu5j2asFryxsxjeoPEuksJc9kwapZUkp...,i want be best
3,e5EfkZKRbASb7ccr7lw08U5qdoEU27Shd_srYxDRMy46VS...,7171771717171771
4,mIZXOAz4h3djNf2QMnxBjXjWAUhMX3tlhREAyiw_Q7nl3R...,Paresz14
...,...,...
95,sRKPoLAq1gHUtHekK-sDKJ4VwvjnxCblF-ZNxM-SCgS9sX...,superior being69
96,YMUlNCf_2BneP7DGow1yGoSFv1yIgmfPj4QspnO1rSmerx...,Camilo
97,OBxv5VC-gQVhr4OA28nnxg-dyfcflqht2lgRRDHWLa4UJg...,suki rzondze
98,ZLuJAuS-TkqscEci1oPtKBBpWuejv9XfSVo8oFiRqoPiAS...,Hobbi


In [91]:
top100 = pd.merge(names, st4, left_on='puuid', right_on='id', how='inner')

In [92]:
top100

,puuid,summoner_name,id,ELO,matches_played,wins,losses
0,t0sKhAPAUcRuPW9A_gIZw_BAanp9nwNNasj8OI9LtT-Wwt...,Δ Desperate,t0sKhAPAUcRuPW9A_gIZw_BAanp9nwNNasj8OI9LtT-Wwt...,1302.253013,197.0,121.0,76.0
1,tQW03KY3EiaFeNRxf_jGkur_9JCNPTQleq9YizxhzMPDhq...,Graaland,tQW03KY3EiaFeNRxf_jGkur_9JCNPTQleq9YizxhzMPDhq...,1749.011626,100.0,66.0,34.0
2,XXpYzb6TNH3GYwSu5j2asFryxsxjeoPEuksJc9kwapZUkp...,i want be best,XXpYzb6TNH3GYwSu5j2asFryxsxjeoPEuksJc9kwapZUkp...,1828.515411,217.0,161.0,56.0
3,e5EfkZKRbASb7ccr7lw08U5qdoEU27Shd_srYxDRMy46VS...,7171771717171771,e5EfkZKRbASb7ccr7lw08U5qdoEU27Shd_srYxDRMy46VS...,1713.535593,156.0,114.0,42.0
4,mIZXOAz4h3djNf2QMnxBjXjWAUhMX3tlhREAyiw_Q7nl3R...,Paresz14,mIZXOAz4h3djNf2QMnxBjXjWAUhMX3tlhREAyiw_Q7nl3R...,1192.477427,96.0,55.0,41.0
...,...,...,...,...,...,...,...
93,HtSK7w5GAdP6bYkXfA2_F-sOXtsvU17fYzhTilOAmLElRr...,east speedrun,HtSK7w5GAdP6bYkXfA2_F-sOXtsvU17fYzhTilOAmLElRr...,974.714332,5.0,2.0,3.0
94,sRKPoLAq1gHUtHekK-sDKJ4VwvjnxCblF-ZNxM-SCgS9sX...,superior being69,sRKPoLAq1gHUtHekK-sDKJ4VwvjnxCblF-ZNxM-SCgS9sX...,1371.513591,19.0,13.0,6.0
95,YMUlNCf_2BneP7DGow1yGoSFv1yIgmfPj4QspnO1rSmerx...,Camilo,YMUlNCf_2BneP7DGow1yGoSFv1yIgmfPj4QspnO1rSmerx...,861.937384,12.0,5.0,7.0
96,OBxv5VC-gQVhr4OA28nnxg-dyfcflqht2lgRRDHWLa4UJg...,suki rzondze,OBxv5VC-gQVhr4OA28nnxg-dyfcflqht2lgRRDHWLa4UJg...,1047.911549,24.0,13.0,11.0


In [93]:
top100 = top100.sort_values(by='ELO', ascending=False)

In [86]:
top100['matches_played'] = round(top100['matches_played'] * 1.2)

In [94]:
top100[['summoner_name', 'ELO', 'matches_played']].to_json('st4.json', orient='records')